In [1]:
import asyncio
import gzip
import json
import logging
import os
import sys
import urllib
import warnings
from datetime import datetime

import dotenv
import pandas as pd
import requests
from sqlalchemy import create_engine, text

### Smoke Test For Model Health Check Point

In [4]:
import importlib.util

path = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_model.py"
spec = importlib.util.spec_from_file_location("rerun_model", path)
rerun_model = importlib.util.module_from_spec(spec)
spec.loader.exec_module(rerun_model)

print("Model version:", rerun_model.get_model_version())


Model version: 16.15.6


## Start of ND volume Analysis

In [ ]:
from sqlalchemy import create_engine 
import urllib
server = '192.168.1.15'
username = 'Junchen'
password = '9DFXjhhlR3k5'
database = 'LF_LMSMaster'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server},1433;DATABASE={database};UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


cnxn = engine.connect()
query1 = """ 

USE LF_LMSMASTER
drop table if EXISTS #t1
select A.Application_ID,A.ApplicationSteps, A.DenialCode, A.PortfolioID, A.CustomerID, A.CustomerSSN ,A.ApplicationDate, CustEmail,
case when ApplicationSteps like '%S%' then 1 else 0 end as NewlyScored,
case when A.ApplicationStatus in ('A','P') then 1 else 0 end as Accepted,
case when A.ApplicationStatus in ('A','P') then A.LeadPurchasePrice else 0 end as LeadPurchasePrice,
case when L.LoanStatus not in ('V','W','G','K') and A.ApplicationStatus='J'  then 1 else 0 end as PartialPreApproved,
case when L.LoanStatus not in ('V','W','G','K')  then 1 else 0 end as Originated,
L.LoanID, datediff(year, VW.DOB, A.ApplicationDate) as Age,
case when VW.Frequency in ('B','S') then 'B' else VW.Frequency end as Frequency,
case when L.LoanStatus not in ('V','W','G','K') then L.OriginatedAmount else null end as OriginatedAmount,
LP.Provider_name, OriginationDate, LOANSTATUS,
case when ((L.RenewalLoanId <> '0') or (A.LPCampaign = 'RENEW'))  then 'RENEWAL' when ApplicationSteps not like '%R%' then 'NEW' else 'REPEAT' end as CustType, RenewalLoanId
into #t1
from Application A
left join Loans L on A.Application_ID = L.ApplicationID and A.PortFolioID = L.PortFolioID
left join LeadProvider LP on A.LeadProviderID = LP.LeadProviderID
left join LF_LMS_Logs..VW_ApplicationDump VW on A.APPGUID = VW.APPGUID
--left join [LF_BankData].[dbo].[IBVStatus] as ibvT on ibvT.AccountNumber = A.CustomerSSN
--left join [LF_BankData].[dbo].[IBVStatus] as ibvT on ibvT.ApplicationID = A.Application_ID AND ibvT.PortfolioID= A.PortfolioID
where A.ApplicationDate>='2025-04-25' and ((CustEmail is null) or  (CustEmail not in ( 'josh@concordadvice.com', 'tiffany.speedyservicing@gmail.com','bobby@speedyservicing.com',
'and.kom@example.com', 'and.tor020@example.com', 'and.tor050@example.com', 'and.torrc4@example.com',
'test@dot818.com', 'test@example.com', 'test@gmail.com', 'test@loanscanada.ca', 'test@test.com', 'test2@test.com',
'testerqad@gmail.com', 'testik@test.com', 'testteam@dmaassociatescorp.com', 'tiffany.speedyservicing@gmail.com')))
order by A.ApplicationDate asc


UPDATE #t1 -- update the renewal loanid from 0 to correct previous loanIDs
SET #t1.RenewalLoanId = B.RenewalLoanId
from #t1 A inner join (select * from
(select A.LoanID, L.LoanID as RenewalLoanId, row_number() over (partition by A.LoanID order by datediff(day, L.OriginationDate, A.OriginationDate) desc) as RN from #t1 A
inner join Application A2 on A.CustomerID = A2.CustomerID and A2.ApplicationDate < A.ApplicationDate
inner join Loans L on A2.Application_ID = L.ApplicationID and A2.PortFolioID = L.PortFolioID and L.OriginationDate < A.OriginationDate and L.LoanStatus not in ('W','V','G','K')
where A.CustType = 'RENEWAL' and A.RenewalLoanId = '0') K where RN=1) B on A.LoanID = B.LoanID


drop table if EXISTs #t11
select rank() over(partition by A.CustomerID order by OriginationDate asc) as CustSSNLoanNum,A.*
into #t11
from #t1 A
--where A.Originated=1 -- 24565 originated Loonie customers (12/23/2025)

-- scoring result for PlaidUDW_v1 and NegativeDBModelLP_v1
drop table if EXISTS #t2
select *
into #t2 from
(select A.*, B1.ExtResponse as B1ExtResponse, B2.ExtResponse as B2ExtResponse,
NDScore, IBVScore, NDBand,IBVBand,
row_number() over (partition by A.Application_ID, A.PortfolioID order by case when B2.ExtResponse is not null and B1.ExtResponse <> '' then 1 else 0 end desc,B.ApplicationDate desc) as RN
from #t11 A
left join Application B on A.CustomerID = B.CustomerID and A.ApplicationDate >= B.ApplicationDate AND B.ApplicationSteps like '%S%'
left join ScoringPythonResult B1 CROSS APPLY OPENJSON(B1.ExtResponse) WITH (IBVScore int '$.ModelScore', IBVBand int '$.IBVBand')  on B.Application_ID = B1.ApplicationID and B.PortFolioID = B1.PortfolioID and B1.iLabel= 'IBVBand'
left join ScoringPythonResult B2 CROSS APPLY OPENJSON(B2.ExtResponse) WITH (NDScore int '$.ModelScore', NDBand int '$.NDBand') on B.Application_ID = B2.ApplicationID and B.PortFolioID = B2.PortfolioID and B2.iLabel= 'NDBand' ) K where RN=1


ALTER TABLE #t2
DROP COLUMN RN;

"""
query2 = """
SELECT *
FROM #t2
ORDER BY ApplicationDate DESC
"""

with engine.begin() as cnxn:  # same connection + transaction
    cnxn.exec_driver_sql("SET NOCOUNT ON; USE LF_LMSMASTER;")
    cnxn.exec_driver_sql(query1)               # multi-statement OK here
    df_perf_orig = pd.read_sql(query2, cnxn)        # sees temp tables
# df_perf = pd.read_csv('perf_accept_orig_formodeling_dependentmeasure.csv')
df_perf_orig = df_perf_orig.rename(columns = {'Application_ID':'AppID'})
df_perf_orig.AppID = df_perf_orig.AppID.astype(int)

print(df_perf_orig.shape)

(80746, 29)


In [9]:
df_perf_orig.columns

Index(['CustSSNLoanNum', 'AppID', 'ApplicationSteps', 'DenialCode',
       'PortfolioID', 'CustomerID', 'CustomerSSN', 'ApplicationDate',
       'CustEmail', 'NewlyScored', 'Accepted', 'LeadPurchasePrice',
       'PartialPreApproved', 'Originated', 'LoanID', 'Age', 'Frequency',
       'OriginatedAmount', 'Provider_name', 'OriginationDate', 'LOANSTATUS',
       'CustType', 'RenewalLoanId', 'B1ExtResponse', 'B2ExtResponse',
       'NDScore', 'IBVScore', 'NDBand', 'IBVBand'],
      dtype='object')

In [10]:
df_perf_orig.head()

,CustSSNLoanNum,AppID,ApplicationSteps,DenialCode,PortfolioID,CustomerID,CustomerSSN,ApplicationDate,CustEmail,NewlyScored,...,OriginationDate,LOANSTATUS,CustType,RenewalLoanId,B1ExtResponse,B2ExtResponse,NDScore,IBVScore,NDBand,IBVBand
0,1,148057,RURU,0,1,42071,51591,2026-01-11 18:57:52.000,None,0,...,NaT,G,RENEWAL,I32513-0,"{""ModelScore"":722,""IBVBand"":4}","{""ModelScore"":676,""NDBand"":3}",676.0,722.0,3.0,4.0
1,4,148056,RURU,0,1,46992,57152,2026-01-11 18:30:49.000,None,0,...,2026-01-11 18:34:06.173,G,RENEWAL,I33733-0,"{""ModelScore"":746,""IBVBand"":4}","{""ModelScore"":999,""NDBand"":6}",999.0,746.0,6.0,4.0
2,1,148055,UUS,34,1,74359,85661,2026-01-11 18:01:36.920,jpichette752@gmail.com,1,...,NaT,None,NEW,None,"{""ModelScore"":613,""IBVBand"":2}","{""ModelScore"":620,""NDBand"":3}",620.0,613.0,3.0,2.0
3,1,148054,UU,43,1,6463,6762,2026-01-11 16:43:15.063,epeitz217@gmail.com,0,...,NaT,None,NEW,None,None,"{""ModelScore"":581,""NDBand"":3}",581.0,NaN,3.0,NaN
4,1,148053,UUS,0,1,64218,74954,2026-01-11 16:30:18.000,rhone_ganda@yahoo.com,1,...,NaT,G,NEW,0,"{""ModelScore"":740,""IBVBand"":4}","{""ModelScore"":568,""NDBand"":2}",568.0,740.0,2.0,4.0


In [6]:
# Quick check: are applications unique in df_perf_orig?
# In this dataset, an "application" should be uniquely identified by (AppID, PortfolioID)

app_keys = ["AppID", "PortfolioID"]

n_rows = len(df_perf_orig)
n_app_pairs = df_perf_orig[app_keys].drop_duplicates().shape[0]
n_dup_app_pairs = df_perf_orig.duplicated(app_keys).sum()

print("rows:", n_rows)
print("unique (AppID, PortfolioID):", n_app_pairs)
print("duplicate rows on (AppID, PortfolioID):", n_dup_app_pairs)

# AppID alone may repeat across portfolios; this tells you if AppID is globally unique
print("unique AppID:", df_perf_orig["AppID"].nunique())
print("unique PortfolioID:", df_perf_orig["PortfolioID"].nunique())

rows: 80746
unique (AppID, PortfolioID): 80746
duplicate rows on (AppID, PortfolioID): 0
unique AppID: 80746
unique PortfolioID: 1


In [16]:
# Application-level ND knockout (application view)
# Compare Raw apps vs NEW+NewlyScored apps.

app_keys = ["AppID", "PortfolioID"]
cutoff = 560  # change if needed

def nd_knockout_summary(df, label):
    d = df.copy()
    d["ApplicationDate"] = pd.to_datetime(d["ApplicationDate"], errors="coerce")
    d["NewlyScored"] = pd.to_numeric(d["NewlyScored"], errors="coerce")
    d["NDScore"] = pd.to_numeric(d["NDScore"], errors="coerce")

    d = (
        d.sort_values(app_keys + ["ApplicationDate"], ascending=[True, True, False])
         .drop_duplicates(app_keys, keep="first")
    )

    n_apps = len(d)
    n_scored = d["NDScore"].notna().sum()
    n_missing = d["NDScore"].isna().sum()
    n_fail = (d["NDScore"].notna() & (d["NDScore"] < cutoff)).sum()
    n_pass = (d["NDScore"].notna() & (d["NDScore"] >= cutoff)).sum()

    return pd.Series({
        "population": label,
        "apps_in_population": n_apps,
        "apps_with_NDScore": n_scored,
        "apps_missing_NDScore": n_missing,
        f"apps_fail_NDScore_lt_{cutoff}": n_fail,
        f"apps_pass_NDScore_ge_{cutoff}": n_pass,
        "fail_rate_among_scored": (n_fail / n_scored) if n_scored else None,
        "fail_rate_overall": (n_fail / n_apps) if n_apps else None,
    })

raw_summary = nd_knockout_summary(df_perf_orig, "Raw apps")
new_summary = nd_knockout_summary(
    df_perf_orig[(df_perf_orig["CustType"] == "NEW") & (pd.to_numeric(df_perf_orig["NewlyScored"], errors="coerce") == 1)],
    "NEW+NewlyScored apps",
)

pd.DataFrame([raw_summary, new_summary])

,population,apps_in_population,apps_with_NDScore,apps_missing_NDScore,apps_fail_NDScore_lt_560,apps_pass_NDScore_ge_560,fail_rate_among_scored,fail_rate_overall
0,Raw apps,80746,70988,9758,7880,63108,0.111005,0.097590
1,NEW+NewlyScored apps,35094,35063,31,2984,32079,0.085104,0.085029


### ND + DenialCode sanity checks (NEW + NewlyScored)

Below we define two commonly-used “populations” so we don’t mix customer-level and application-level counts:

- **Application-level view**: unique application key = (`AppID`, `PortfolioID`)  
- **Customer-level view** (your earlier approach): 1 row per `CustomerID` (earliest NEW+Scored application per customer)


In [17]:
# Helper: define consistent populations + clean types

app_keys = ["AppID", "PortfolioID"]
cutoff = 560  # ND cutoff

_df = df_perf_orig.copy()
_df["ApplicationDate"] = pd.to_datetime(_df["ApplicationDate"], errors="coerce")

# DenialCode sometimes comes in as numeric; normalize to a clean string label
# We'll keep NULLs as <NA> (pandas) so value_counts(dropna=False) is informative.
_df["DenialCode"] = (
    _df["DenialCode"]
      .astype("Int64", errors="ignore")
      .astype(str)
      .replace({"<NA>": pd.NA, "nan": pd.NA, "None": pd.NA})
)

for col in ["NewlyScored", "NDScore", "IBVScore", "NDBand", "IBVBand"]:
    if col in _df.columns:
        _df[col] = pd.to_numeric(_df[col], errors="coerce")

# Raw view (no filter). Safety: dedup by app key if ever needed.
raw_apps = (
    _df.sort_values(app_keys + ["ApplicationDate"], ascending=[True, True, False])
       .drop_duplicates(app_keys, keep="first")
       .copy()
)

# NEW + NewlyScored view (application-level)
new_base = _df[(_df["CustType"] == "NEW") & (_df["NewlyScored"] == 1)].copy()
new_apps = (
    new_base.sort_values(app_keys + ["ApplicationDate"], ascending=[True, True, False])
            .drop_duplicates(app_keys, keep="first")
            .copy()
)

print("Raw perf rows:", len(df_perf_orig))
print("Raw apps (dedup by AppID+PortfolioID):", len(raw_apps))
print("NEW+NewlyScored rows:", len(new_base))
print("NEW+NewlyScored apps (dedup by AppID+PortfolioID):", len(new_apps))

Raw perf rows: 80746
Raw apps (dedup by AppID+PortfolioID): 80746
NEW+NewlyScored rows: 35094
NEW+NewlyScored apps (dedup by AppID+PortfolioID): 35094


In [18]:
# Q1) For rows with missing NDScore (ND fields null), what DenialCode values do we see?
# Compare Raw apps vs NEW+NewlyScored apps.

def denialcode_breakdown(df, label):
    missing_nd = df[df["NDScore"].isna()].copy()
    print("\n---", label, "---")
    print("rows:", len(df))
    print("missing NDScore rows:", len(missing_nd))
    print("missing NDScore rate:", (len(missing_nd) / len(df)) if len(df) else None)
    print("\nDenialCode counts for missing NDScore (top 20):")
    display(missing_nd["DenialCode"].value_counts(dropna=False).head(20))

    print("\nDenialCode share for missing NDScore (top 20):")
    display((missing_nd["DenialCode"].value_counts(dropna=False, normalize=True).head(20) * 100).round(2))


denialcode_breakdown(raw_apps, "Raw apps (all rows, dedup by AppID+PortfolioID)")
denialcode_breakdown(new_apps, "NEW+NewlyScored apps (dedup by AppID+PortfolioID)")


--- Raw apps (all rows, dedup by AppID+PortfolioID) ---
rows: 80746
missing NDScore rows: 9758
missing NDScore rate: 0.12084809154633047

DenialCode counts for missing NDScore (top 20):


DenialCode
43     3356
113    1605
18     1021
13      896
100     845
117     428
30      414
6       236
56      220
11      198
118     100
15       83
45       70
110      68
46       55
0        52
44       38
19       23
5        17
34       15
Name: count, dtype: int64


DenialCode share for missing NDScore (top 20):


DenialCode
43     34.39
113    16.45
18     10.46
13      9.18
100     8.66
117     4.39
30      4.24
6       2.42
56      2.25
11      2.03
118     1.02
15      0.85
45      0.72
110     0.70
46      0.56
0       0.53
44      0.39
19      0.24
5       0.17
34      0.15
Name: proportion, dtype: float64


--- NEW+NewlyScored apps (dedup by AppID+PortfolioID) ---
rows: 35094
missing NDScore rows: 31
missing NDScore rate: 0.0008833418818031572

DenialCode counts for missing NDScore (top 20):


DenialCode
34     15
0      13
115     3
Name: count, dtype: int64


DenialCode share for missing NDScore (top 20):


DenialCode
34     48.39
0      41.94
115     9.68
Name: proportion, dtype: float64

In [19]:
# Extra: among missing NDScore rows, how many are DenialCode=34 specifically?

def missing_nd_denial34(df, label):
    m = df[df["NDScore"].isna()].copy()
    n = len(m)
    n34 = (m["DenialCode"] == "34").sum()
    pct = (n34 / n * 100) if n else None
    print(f"{label}: missing NDScore={n}, DenialCode=34 among missing NDScore={n34}, pct={pct:.2f}%" if n else f"{label}: missing NDScore=0")

missing_nd_denial34(raw_apps, "Raw apps")
missing_nd_denial34(new_apps, "NEW+NewlyScored apps")

Raw apps: missing NDScore=9758, DenialCode=34 among missing NDScore=15, pct=0.15%
NEW+NewlyScored apps: missing NDScore=31, DenialCode=34 among missing NDScore=15, pct=48.39%


In [20]:
# Q2) DenialCode=34: split into ND-driven vs downstream (ND ok / missing)
# We treat "ND-driven" as DenialCode=34 AND NDScore < cutoff.
# Compare Raw apps vs NEW+NewlyScored apps.

import numpy as np

def denial34_nd_split(df, label):
    d = df.copy()

    nd_bucket = np.select(
        [d["NDScore"].isna(), d["NDScore"] < cutoff, d["NDScore"] >= cutoff],
        ["NDScore_MISSING", f"NDScore_LT_{cutoff}", f"NDScore_GE_{cutoff}"],
        default="NDScore_OTHER",
    )
    d["ND_bucket"] = nd_bucket

    d34 = d[d["DenialCode"] == "34"].copy()

    print("\n---", label, "---")
    print("Total rows:", len(d))
    print("DenialCode=34 rows:", len(d34))

    print("\nDenialCode=34 split by ND bucket:")
    display(d34["ND_bucket"].value_counts(dropna=False))

    print("\nDenialCode=34 split by ND bucket (%):")
    display((d34["ND_bucket"].value_counts(normalize=True) * 100).round(2))

    # Optional downstream check: does IBVScore exist on these?
    print("\nWithin DenialCode=34, IBVScore present rate by ND bucket:")
    tmp = d34.assign(IBV_present=d34["IBVScore"].notna())
    display(tmp.groupby("ND_bucket")["IBV_present"].mean().mul(100).round(2))


denial34_nd_split(raw_apps, "Raw apps")
denial34_nd_split(new_apps, "NEW+NewlyScored apps")


--- Raw apps ---
Total rows: 80746
DenialCode=34 rows: 16407

DenialCode=34 split by ND bucket:


ND_bucket
NDScore_GE_560     13810
NDScore_LT_560      2582
NDScore_MISSING       15
Name: count, dtype: int64


DenialCode=34 split by ND bucket (%):


ND_bucket
NDScore_GE_560     84.17
NDScore_LT_560     15.74
NDScore_MISSING     0.09
Name: proportion, dtype: float64


Within DenialCode=34, IBVScore present rate by ND bucket:


...

ND_bucket
NDScore_GE_560     13810
NDScore_LT_560      2582
NDScore_MISSING       15
Name: count, dtype: int64


DenialCode=34 split by ND bucket (%):


ND_bucket
NDScore_GE_560     84.17
NDScore_LT_560     15.74
NDScore_MISSING     0.09
Name: proportion, dtype: float64


Within DenialCode=34, IBVScore present rate by ND bucket:


ND_bucket
NDScore_GE_560     99.99
NDScore_LT_560     17.27
NDScore_MISSING     0.00
Name: IBV_present, dtype: float64